# making FST


In [1]:
states={'EA','MEA','MEA1','MEA2','MEA3','MYNR','MYNR1','MYR1','Reject','YNR','YR','i','q0','s'}
input_symbols=set([chr(c) for c in range(ord('a'), ord('z')+1)])
start_state='q0'
final_states={'EA','MEA','MEA1','MEA2','MEA3','MYNR','MYNR1','MYR1','MYR','YNR','YR','i','q0','s'}

In [2]:
# input_symbols

# table

In [3]:
vowels=['a','e','o','u']
special=['z','x','s','c']
transition_table={}

In [4]:
# for  y replacement

transition_table[('q0','i')]='i'
transition_table[('i','s')]='Reject'
transition_table[('i','e')]='MYR1'
transition_table[('MYR1','s')]='YR'

for c in input_symbols:
    if c!='s' and c!='e':
        transition_table[('MYR',c)]='q0'

for c in input_symbols:
    if c!='s':
        transition_table[('MYR1',c)]='q0'

for c in input_symbols:
    transition_table[('YR',c)]='q0'


In [5]:
# for y not replaced
for c in vowels:
    transition_table[('q0', c)] = 'MYNR'

transition_table[('i','y')] = 'MYNR1'
transition_table[('MYNR', 'y')]='MYNR1'
transition_table[('MYNR1', 's')]='YNR'

for c in input_symbols:
    if c!='s':
        transition_table[('MYNR1',c)]='q0'

for c in input_symbols:
    transition_table[('YNR',c)]='q0'

In [6]:
# insertion of e
transition_table[('q0','s')]='s'
# transition_table[('s','e')]='MEA2'
transition_table[('s','h')]='MEA3'
transition_table[('s','s')]='MEA'


transition_table[('q0','x')]='MEA'
transition_table[('q0','z')]='MEA'
transition_table[('q0','c')]='MEA1'
transition_table[('MEA1','h')]='MEA3'
transition_table[("MEA"),'s']='Reject'

transition_table[('MEA','e')]='MEA2'
transition_table[('MEA3','e')]='MEA2'

transition_table[('MEA2','s')]='EA'

transition_table[('MEA3','s')]='Reject'
transition_table[('MEA','s')]='Reject'

for  c in input_symbols:
    if c!='e' and c!='s':
        transition_table[('MEA',c)]='q0'


for  c in input_symbols:
    if c!='h':
        transition_table[('MEA1',c)]='q0'

for  c in input_symbols:
    if c!='e' and c!='s':
        transition_table[('MEA3',c)]='q0'


for c in input_symbols:
    transition_table[('EA',c)]='q0'

In [7]:
# only s
for c in input_symbols:
    if c!='s' and c!='h':
        transition_table[('s',c)]='q0'

for s in ['MEA3','MYNR','q0']:
    transition_table[(s,'s')]='Reject'

for s in ['MEA1','MEA2','MEA3','MYNR','q0']:
    transition_table[(s,'s')]='s'


In [8]:
# to q0

for c in input_symbols:
    if c not in vowels and c not in ['i','y','s','z','x','c','h','e']:
        transition_table[('q0', c)] = 'q0'

for c in input_symbols:
    if c not in vowels and c not in ['i','y','s','z','x','c','h','e']:
        for s in final_states:
            transition_table[(s, c)] = 'q0'

In [9]:
set(transition_table.values())

{'MEA',
 'MEA1',
 'MEA2',
 'MEA3',
 'MYNR',
 'MYNR1',
 'MYR1',
 'Reject',
 'YNR',
 'YR',
 'i',
 'q0',
 's'}

# running FST

In [10]:
def  morph(word):
    curr_state='q0'

    for c in word:
        if c in input_symbols:
            curr_state = transition_table.get((curr_state, c), 'q0')

     
    # return curr_state
    if curr_state=='s':
        morph_of_w=word[:-1]+'+N+PL'
    elif curr_state=='EA':
        morph_of_w=word[:-2]+'+N+PL'
    elif curr_state=='YR':
        morph_of_w=word[:-3]+'y+N+PL'   
    elif curr_state=='YNR':
        morph_of_w=word[:-1]+'+N+PL'
    elif curr_state=='Reject':
        morph_of_w="Invalid world"
    elif curr_state in ['MEA','MEA1','MEA2','MEA3','MYNR','MYNR1','MYR1','i','q0']:
        morph_of_w=word+"+N+SG"+" " 
    else:
        morph_of_w=word+curr_state    

    return morph_of_w      



# runing

In [11]:
words = ["parties", "buses", "dogs","foxs", "foxs","foxes", "watch", "apples","pens"]

for w in words:
    print(morph(w))

party+N+PL
buse+N+PL
dog+N+PL
fox+N+PL
fox+N+PL
foxe+N+PL
watch+N+SG 
apple+N+PL
pen+N+PL


# processing txt file

In [12]:
# File paths
input_file = "brown_nouns.txt"
output_file = "DFA_fst_output.txt"

with open(input_file, "r", encoding="utf-8") as f:
    words = [line.strip() for line in f if line.strip()]

results = []
for word in words:
    results.append(word+"="+morph(word))

# Save results
with open(output_file, "w", encoding="utf-8") as f:
    f.write("\n".join(results))

print(f"✅ FST processing complete! Output saved to {output_file}")

✅ FST processing complete! Output saved to DFA_fst_output.txt


In [13]:
def nfa_simulate(word, transition_table, start_state, final_states):
    # current states is a *set* because NFA can be in multiple states
    current_states = {start_state}
    
    for symbol in word:
        next_states = set()
        for state in current_states:
            if (state, symbol) in transition_table:
                for target in transition_table[(state, symbol)]:
                    next_states.add(target)
        current_states = next_states
    
    # Accept if any of the current states is a final state
    return any(state in final_states for state in current_states)


# Example usage:
states = {}
input_symbols = set([chr(c) for c in range(ord('a'), ord('z')+1)])
start_state = 'q0'
final_states = {'MYR', 'MYNR'}   # ✅ must define final states
transition_table = {}

# Add transitions
transition_table[('q0','i')] = ['MYR','MYNR']
for c in ['a','e','o','u']:
    transition_table[('q0', c)] = ['MYNR']

# Test words
words = ["i", "a", "u", "z"]
for w in words:
    if nfa_simulate(w, transition_table, start_state, final_states):
        print(f"{w}: ✅ Valid Word")
    else:
        print(f"{w}: ❌ Invalid Word")


i: ✅ Valid Word
a: ✅ Valid Word
u: ✅ Valid Word
z: ❌ Invalid Word


# FST visulization

In [14]:
from graphviz import Digraph
import string

In [15]:


def simplify_label(symbols):
    """Convert full a-z list to a short 'a–z' string if possible."""
    if set(symbols) == set(string.ascii_lowercase):
        return 'a–z'
    return ', '.join(sorted(symbols))

def visualize_dfa_clean(states, input_symbols, transition_table, start_state, accept_states, filename='clean_dfa'):
    dot = Digraph(format='png')
    dot.attr(rankdir='LR')  # Left to right layout

    # Draw states
    for state in states:
        if state in accept_states:
            dot.node(state, shape='doublecircle')
        elif 'reject' in state:
            dot.node(state, shape='circle', style='filled', fillcolor='lightcoral')
        else:
            dot.node(state, shape='circle')

    # Fake invisible start arrow
    dot.node('', shape='none')
    dot.edge('', start_state)

    # Collect edges: (src, dst) → list of input symbols
    edge_labels = {}
    for (src, symbol), dst in transition_table.items():
        key = (src, dst)
        edge_labels.setdefault(key, []).append(symbol)

    # Draw transitions with grouped labels
    for (src, dst), symbols in edge_labels.items():
        label = simplify_label(symbols)
        dot.edge(src, dst, label=label)

    # Render and open
    dot.render(filename, view=True)



def visualize_dfa_individual_arrows(states, input_symbols, transition_table, start_state, accept_states, filename='dfa_individual_arrows'):
    dot = Digraph(format='png')
    dot.attr(rankdir='LR')  # Left to right layout

    # Draw states
    for state in states:
        if state in accept_states:
            dot.node(state, shape='doublecircle')
        elif 'reject' in state:
            dot.node(state, shape='circle', style='filled', fillcolor='lightcoral')
        else:
            dot.node(state, shape='circle')

    # Start arrow
    dot.node('', shape='none')
    dot.edge('', start_state)

    # Draw individual transitions (no grouping)
    for (src, symbol), dst in transition_table.items():
        dot.edge(src, dst, label=symbol)

    # Render
    dot.render(filename, view=True)





In [16]:
# Visualize with individual arrows
# visualize_dfa_individual_arrows(states, input_symbols, transition_table, start_state, final_states)


# Visualize
visualize_dfa_clean(states, input_symbols, transition_table, start_state, final_states,"DFA_fst")

TypeError: unhashable type: 'list'